
**[PT]** Português

---

**[EN]** English


# Correção, inferência e normalização do campo "Faculdade"


---


# Correction, inference and normalization of the field "Faculdade"



## Setup

In [1]:
from timelink.api.database import TimelinkDatabase
from timelink import version
from ucalumni.config import default_db_url

print(f"Timelink API version: {version}")
print(f"Creating TimelinkDatabase instance from {default_db_url}")
db = TimelinkDatabase(db_url=default_db_url)


Timelink API version: 1.1.14
Creating TimelinkDatabase instance from sqlite:///../database/sqlite3/fauc2.db?check_same_thread=False


## Notas de processamento

As notas de processamento são criadas pelo algoritmo de extração do valor da Faculdade.

Contém informação sobre inferências, correções automáticas e erros que necessitam de revisão.

Em termos narrativos, o algoritmo verifica primeiro se o campo "Faculdade" tem ou não um valor; depois procede à recolha de todas as referências a faculdades no resto do registo; se ao remover os pré-requisitos da recolha algo permanece, será o valor escolhido como faculdade; se nada permanece, então o valor original é usado; alguns outros casos especiais são testados. 


--- 

## Processing notes

The processing notes are created to register inferences, corrections and errors that need revision.

In narrative terms, the algorithm checks first if the field “Faculdade” has a value or not; it then proceeds to collect all references to “faculdades” in the rest of the record; if by removing the pre-requisites from the collection something remains, it will the chosen as the corrected or inferred “faculdade”; if nothing remains then the original value is used; some other edge cases are tested. 




In [5]:
df = notas_proc
df['id'] = df.index.values

col = 'nota-processamento' # sub total by this column
df_totals = df.groupby(col).agg({'id':'nunique',
                                                  'uc-entrada':'min',
                                                  'uc-saida':'max'})
df_totals.loc['Total'] = df_totals['id'].sum()

df_totals.sort_values('id',ascending= False).head(30)

,id,uc-entrada,uc-saida
nota-processamento,,,
Total,20066,20066,20066
Aviso: faculdade inferida,11361,0000-00-00,1913-07-11
Aviso: faculdade corrigida,6990,0000-00-00,1916-07-15
Erro: não foi possível determinar a faculdade,674,1537-09-27,1912-08-02
"Faculdade corrigida, combinação incomum",632,0000-00-00,1916-07-15
Erro: valor não corresponde a Faculdade nesta data,409,0000-00-00,1913-08-13



### Faculdade inferida

Casos em que o campo da faculdade não foi preenchido e o algoritmo sugere um valor.

---

### "Faculdade" Inferred


Cases where the value for faculty was missing and the algoritm suggested a value.



In [6]:
fac_inf = notas_proc[notas_proc['nota-processamento']=='Aviso: faculdade inferida'].copy()
fac_inf.dropna(how='all', axis=1, inplace=True)
fac_inf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11765 entries, 140351 to 196148
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   id_1                      11765 non-null  object
 1   name                      11765 non-null  object
 2   nota-processamento.type   11765 non-null  object
 3   nota-processamento        11765 non-null  object
 4   nota-processamento.date   11765 non-null  object
 5   nota-processamento.line   11765 non-null  int64 
 6   nota-processamento.level  11765 non-null  int64 
 7   nota-processamento.obs    11765 non-null  object
 8   faculdade                 11765 non-null  object
 9   faculdade.date            11765 non-null  object
 10  faculdade.obs             11765 non-null  object
 11  uc-entrada                11765 non-null  object
 12  uc-entrada.date           11765 non-null  object
 13  uc-entrada.obs            11765 non-null  object
 14  uc-saida             

In [7]:
df = fac_inf
col = 'faculdade' # subotal by this column
df_totals = df.groupby(col).agg({'id':'nunique',
                                                  'uc-entrada':'min',
                                                  'uc-saida':'max'})
print("List of inferred 'faculdade'")
df_totals.sort_values('id',ascending= False).head(30)

List of inferred 'faculdade'


,id,uc-entrada,uc-saida
faculdade,,,
Cursos jurídicos (Cânones ou Leis),6993,0000-00-00,1829-03-28
Artes,3010,0000-00-00,1780-07-18
Cânones,764,0000-00-00,1827-06-20
Leis,386,0000-00-00,1828-07-01
Teologia,353,0000-00-00,1884-05-26
Medicina,185,0000-00-00,1883-10-15
Filosofia,28,1770-10-01,1913-07-11
Matemática,24,1605-10-07,1913-07-11
Direito,22,1836-10-14,1898-12-18


### Faculdade corrigida

Casos em que o campo da faculdade estava preenchido mas era inconsistente com o resto da informação.

A maior parte dos casos dizem respeito a duplo percursos em Leis e Cânones e mudanças, depois de 1772, 
de faculdades de Matemática e Filosofia, que eram pre-requisitos, para as faculdades de posterior
matrículas (Leis, Cânones, Medicina, Teologia).

---

### "Faculdade" corrected

Cases where the algoritm changed the value of the field, because it was inconsistent with
the rest of information on the record. Mostly double degres in Canon and Civil law, and
changes from pre-requisite faculty to final graduation faculty.


In [8]:
fac_corr = notas_proc[notas_proc['nota-processamento']=='Aviso: faculdade corrigida'].copy()
fac_corr.dropna(how='all', axis=1, inplace=True)

In [9]:
import pandas as pd

df = fac_corr
col = 'nota-processamento.obs' # subotal by this column
df_totals = df.groupby(col).agg({'id':'nunique',
                                                  'uc-entrada':'min',
                                                  'uc-saida':'max'})
total = df_totals['id'].sum()
print("List of corrected 'faculdade'")
print("Total:",total)
df_totals['perc'] = df_totals['id']/total
pd.set_option('display.max_rows',1000)
df_totals.sort_values('id',ascending=False, inplace=True)
df_totals.head(10)

List of corrected 'faculdade'
Total: 6990


,id,uc-entrada,uc-saida,perc
nota-processamento.obs,,,,
"Cânones para Cânones,Leis.",1254,0000-00-00,1821-11-12,0.179399
"Leis para Cânones,Leis.",1246,0000-00-00,1829-06-13,0.178255
Matemática para Medicina.,1177,0000-00-00,1909-11-11,0.168383
Direito para Leis.,1022,0000-00-00,1827-06-16,0.146209
Filosofia para Medicina.,480,0000-00-00,1909-11-09,0.068670
"Teologia para Direito,Teologia.",232,1830-10-11,1915-10-22,0.033190
"Medicina para Cânones,Medicina.",209,1569-11-20,1778-10-20,0.029900
Direito para Cânones.,193,0000-00-00,1835-07-20,0.027611
"Cânones para Cânones,Teologia.",166,0000-00-00,1829-05-26,0.023748


In [10]:
fac_corr_nodup = fac_corr.drop_duplicates(subset=['id', 'faculdade-original'], keep='first')

In [11]:
show_only = 5
for  change,row in list(df_totals.iterrows())[:10]:
    print(f"{change}: {row.id}")
    for _ndx, row in fac_corr_nodup[fac_corr_nodup['nota-processamento.obs']==change][['id','name','url','uc-entrada']].head(show_only).iterrows():
        url = row.url.strip('"')
        print(f"     {row.id:6} {row['name']} {row['uc-entrada']} {url}")


Cânones para Cânones,Leis.: 1254
     140543 António de Abreu 1624-10-26 https://pesquisa.auc.uc.pt/details?id=140543
     140695 Duarte de Abreu 1566-01-20 https://pesquisa.auc.uc.pt/details?id=140695
     140772 Francisco Nunes de Abreu 1677-10-01 https://pesquisa.auc.uc.pt/details?id=140772
     140791 Gaspar de Abreu 1605-10-12 https://pesquisa.auc.uc.pt/details?id=140791
     140847 Valério de Abreu 1624-10-30 https://pesquisa.auc.uc.pt/details?id=140847
Leis para Cânones,Leis.: 1246
     140425 João de Abranches 1729-10-29 https://pesquisa.auc.uc.pt/details?id=140425
     140529 Ambrósio de Abreu 1653-10-15 https://pesquisa.auc.uc.pt/details?id=140529
     140617 António de Sousa de Abreu 1703-10-01 https://pesquisa.auc.uc.pt/details?id=140617
     140780 Francisco da Silva e Abreu 1739-10-01 https://pesquisa.auc.uc.pt/details?id=140780
     140790 Gaspar de Abreu 1610-10-05 https://pesquisa.auc.uc.pt/details?id=140790
Matemática para Medicina.: 1177
     140420 Guilherme da Silv

### "Valor não corresponde a Faculdade nesta data"

Esta lista é produzida pela comparação do valor do campo de faculdade, ou outra referência a faculdade no registo, e a lista das faculdades existentes à data do registo.

Para além dos erros de ortografia e gralhas (atenção a "Cãnones" com til), também contém:

* Uma grande quantidade (>300) de campos de Faculdade com "?". Verificar se é porque falta informação na ficha, ou se existe outra explicação
* valores confundidos com Faculdade por particularidades da pontuação do registo. Nesse caso a correção a fazer pode ser na pontuação. 
   Ver por exemplo: 152975 a falta de ":" a seguir a matrícula provoca um erro no processamento da Faculdade
* valor que correspondem a faculdades que existiram, mas não naquelas datas (por exemplo Filosofia antes de 1772 ou Cânones depois de 1834).
* Topónimos, que serão erros de digitação que se poderão corrigir vendo a ficha original.

---

### Value does not correspond to a "Faculdade" at the time

This list is produced by comparing the value of the faculty field, or other reference to faculties in the record, and the list of faculties existing at the date of the record.

Apart from spelling errors and typos (watch out for tilted "Cãnones"), it also contains:

* A large amount (>300) of Faculty fields with "?". Check if this is because the information is missing in the original card, or if there is another explanation.
* Values confused with Faculty due to particularities of the register punctuation. In this case, the correction to be made may be in the punctuation. 
   See for example: 152975 the lack of ":" after the enrolment causes an error in the processing of the Faculty
* value corresponding to faculties that existed but not on those dates (e.g. Philosophy before 1772 or "Cânones" after 1834).
* Toponyms, which are typing errors that can be corrected by looking at the original form.

In [12]:
fac_error = notas_proc[notas_proc['nota-processamento']=='Erro: valor não corresponde a Faculdade nesta data'].copy()
fac_error.info()

<class 'pandas.core.frame.DataFrame'>
Index: 409 entries, 140542 to 317873
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   id_1                      409 non-null    object
 1   name                      409 non-null    object
 2   nota-processamento.type   409 non-null    object
 3   nota-processamento        409 non-null    object
 4   nota-processamento.date   409 non-null    object
 5   nota-processamento.line   409 non-null    int64 
 6   nota-processamento.level  409 non-null    int64 
 7   nota-processamento.obs    409 non-null    object
 8   faculdade                 409 non-null    object
 9   faculdade.date            409 non-null    object
 10  faculdade.obs             409 non-null    object
 11  faculdade-original        98 non-null     object
 12  faculdade-original.date   98 non-null     object
 13  faculdade-original.obs    98 non-null     object
 14  uc-entrada             

In [13]:
# currently the algorithm removed question marks, but they were there
# we have to put them back until this is fixed
fac_org_none = fac_error['faculdade-original'].isnull()
fac_error.loc[fac_org_none,'faculdade-original'] = '?'


In [14]:
df = fac_error
col = 'faculdade-original' # subotal by this column
df_totals = df.groupby(col).agg({'id':'nunique',
                                                  'uc-entrada':'min',
                                                  'uc-saida':'max'})

df_totals.sort_values('id',ascending= False).head(30)

,id,uc-entrada,uc-saida
faculdade-original,,,
?,311,0000-00-00,1771-01-01
Leis,23,1827-10-13,1909-08-03
Cânones,22,1825-10-24,1913-08-13
.,13,1604-10-17,1732-11-10
Intituta,7,1581-03-12,1770-10-01
Instituto,7,1694-12-15,1818-10-24
Dialética,4,1540-03-09,1542-03-06
Institua,4,1642-10-28,1733-10-01
Estado Eclesiástico,3,1846-11-11,1865-10-11


In [15]:
import pandas as pd
import csv

pd.set_option('display.max_rows',600)
exportar = fac_error[['faculdade','name','faculdade.date']].sort_values(['faculdade','name','faculdade.date'])
exportar.to_csv('../inferences/validation/faculdade-unkown.csv',sep=',')
exportar.head(10)

,faculdade,name,faculdade.date
id,,,
146645,(.),António Antunes,1656-10-20
140542,(.),António de Abreu,1661-10-26
165702,(.),Manuel Ribeiro,1604-10-17
165556,(.),Manuel Ribeiro,1612-03-23
165558,(.),Manuel Ribeiro,1617-04-09
165781,(.),Manuel Ribeiro,1657-10-15
165804,(.),Manuel Ribeiro,1659-12-12
165730,(.),Manuel Ribeiro,1683-03-24
165828,(.),Manuel Ribeiro,1689-11-24


### "Faculdades" com menos de 100 ocorrências. 

---


### "Faculdades" with less than 100 stduents

In [17]:

from timelink.pandas import attribute_values


facs = attribute_values('faculdade',
                        dates_between=('1500-00-00','1990-00-00'),
                        db=db)
facs[facs['count']<100]

,count,date_min,date_max
value,,,
(Leis),33,1819-10-16,1906-10-02
(Cânones),32,1822-10-15,1909-10-02
(.),16,1604-10-17,1738-12-01
código,12,1540-00-00,1540-12-16
(Instituto),7,1694-12-15,1818-10-24
(Intituta),7,1581-03-12,1770-10-01
(Dialética),5,1540-03-09,1540-10-25
(Institua),4,1642-10-28,1733-10-01
(Cãnones),3,1572-10-01,1626-10-06


In [23]:
from timelink.pandas import entities_with_attribute

facs_errors = facs[facs['count']<100]
show_only = 100
for f,row in facs_errors.sort_values('count',ascending=False).iterrows():
        print()
        print(f)
        students = entities_with_attribute(
                entity_type='person',
                the_type='faculdade',
                the_value=f,
                show_elements=['id','name'],
                more_attributes=['nota-processamento'],
                db=db)
        for id, aluno in students.head(show_only).iterrows():
                aviso = aluno["nota-processamento"]
                if aviso is not None:
                        aviso = aviso + " " + aluno['nota-processamento.obs']
                else:
                        aviso = ""
                id_link = f'<a href="https://pesquisa.auc.uc.pt/details?id={id}">{id}</a>'
                # display(HTML(f'<code>{row.the_value:24} [{id_link:5}] {aluno.the_date} {aluno.name:44}</code>'))
                print(f"{aluno['faculdade']:24} [{id:5}] {aluno['faculdade.date']} {aluno['name']:44} {aviso} https://pesquisa.auc.uc.pt/details?id={id} {aluno['faculdade.obs']}")



(Leis)
(Leis)                   [188758] 1819-10-16 Manuel Joaquim de Sousa Brito                Aviso: faculdade corrigida Leis para (Leis),Direito. https://pesquisa.auc.uc.pt/details?id=188758 Faculdade corrigida
(Leis)                   [188758] 1819-10-16 Manuel Joaquim de Sousa Brito                Faculdade corrigida, combinação incomum Leis para (Leis),Direito. https://pesquisa.auc.uc.pt/details?id=188758 Faculdade corrigida
(Leis)                   [215793] 1822-10-08 António Pedro Mousinho Leote                 Aviso: faculdade corrigida Leis para (Leis),Direito. https://pesquisa.auc.uc.pt/details?id=215793 Faculdade corrigida
(Leis)                   [215793] 1822-10-08 António Pedro Mousinho Leote                 Faculdade corrigida, combinação incomum Leis para (Leis),Direito. https://pesquisa.auc.uc.pt/details?id=215793 Faculdade corrigida
(Leis)                   [141618] 1823-10-07 Miguel Luís Henriques de Aguiar              Aviso: faculdade corrigida Leis para (Leis),